# LionAGI - Introduction 12 : Branch Info, Filter, and Edit

In [ ]:
import lionagi as li

def multiply(number1:float, number2:float):
    '''
    Perform multiplication on two numbers.

    Args:
        number1: First number to multiply.
        number2: Second number to multiply.
    
    Returns:
        The product of number1 and number2.

    '''
    return number1*number2

tool_mul = li.func_to_tool(multiply)

# and now some messages
system = "you are asked to perform as a function picker and parameter provider"
task = "Think step by step, understand the following basic math question and provide parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct1 = {"Task": task, "json_format": json_format}

question = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"
question2 = "There are three fruits in total, each with 2 different colors, how many unique kinds of fruits are there?"

context1 = {"Question1": question, "question2": question2}

# by_default, tools are not used, you need to specify 
# tools = True, allows all tools to be available to use
# tools = schema / Tool Object let you to specify which tools are allowed to use

session = li.Session(system=system)
session.register_tools(tool_mul)

await session.chat(instruction=instruct1, context=context1, tools=True, response_format={'type':"json_object"})

{'action_request': [{'recipient_name': 'functions.multiply',
   'parameters': {'number1': 3, 'number2': 4}},
  {'recipient_name': 'functions.multiply',
   'parameters': {'number1': 3, 'number2': 2}}]}

In [ ]:
branch = session.default_branch
session.messages

,node_id,role,sender,timestamp,content
0,bd62dd97064b9053f46f04de136bafc4,system,system,2024-02-02 23:48:27.864540,"{""system_info"": ""you are asked to perform as a..."
1,85111b57a85ba964acc33d44ae83786d,user,user,2024-02-02 23:48:27.865715,"{""instruction"": {""Task"": ""Think step by step, ..."
2,48d4f75218f74f7a3a16c259e6be3bf1,assistant,action_request,2024-02-02 23:48:33.040024,"{""action_request"": [{""recipient_name"": ""functions..."
3,960089cac08d9cf4635f8639db9b1ba6,assistant,action_response,2024-02-02 23:48:33.041690,"{""action_response"": {""function"": ""multiply"", ""..."
4,868b9e265c6e5aa46b767efcc6a45a3d,assistant,action_response,2024-02-02 23:48:33.042977,"{""action_response"": {""function"": ""multiply"", ""..."


### Info

In [ ]:
# complete version
branch.messages_describe

{'total_messages': 5,
 'summary_by_role': {'assistant': 3, 'system': 1, 'user': 1, 'total': 5},
 'summary_by_sender': {'action_response': 2,
  'system': 1,
  'user': 1,
  'action_request': 1,
  'total': 5},
 'instruction_sets': {},
 'registered_tools': {'multiply': Tool({"node_id":"6921e3a322f092b57b45f281fa4dfa8b","metadata":{},"label":null,"related_nodes":[],"content":null,"func":"multiply","parser":null,"schema_":{"type":"function","function":{"name":"multiply","description":"Perform multiplication on two numbers.","parameters":{"type":"object","properties":{"number1":{"type":"number","description":"First number to multiply."},"number2":{"type":"number","description":"Second number to multiply."}},"required":["number1","number2"]}}}})},
 'messages': [{'node_id': 'bd62dd97064b9053f46f04de136bafc4',
   'role': 'system',
   'sender': 'system',
   'timestamp': Timestamp('2024-02-02 23:48:27.864540'),
   'content': '{"system_info": "you are asked to perform as a function picker and param

In [ ]:
branch.info()

{'assistant': 3, 'system': 1, 'user': 1, 'total': 5}

In [ ]:
branch.last_instruction

,node_id,role,sender,timestamp,content
1,85111b57a85ba964acc33d44ae83786d,user,user,2024-02-02 23:48:27.865715,"{""instruction"": {""Task"": ""Think step by step, ..."


In [ ]:
branch.last_response

,node_id,role,sender,timestamp,content
4,868b9e265c6e5aa46b767efcc6a45a3d,assistant,action_response,2024-02-02 23:48:33.042977,"{""action_response"": {""function"": ""multiply"", ""..."


### Filter

In [ ]:
# filter_messages_by(role=None, sender=None, start_time:datetime=None, end_time:datetime=None, content_keywords=None, case_sensitive=False)
branch.filter_messages_by(role='assistant', content_keywords='output')

,node_id,role,sender,timestamp,content
3,960089cac08d9cf4635f8639db9b1ba6,assistant,action_response,2024-02-02 23:48:33.041690,"{""action_response"": {""function"": ""multiply"", ""..."
4,868b9e265c6e5aa46b767efcc6a45a3d,assistant,action_response,2024-02-02 23:48:33.042977,"{""action_response"": {""function"": ""multiply"", ""..."


In [ ]:
# search_keywords(keywords=None, case_sensitive=False)
branch.search_keywords('task')

,node_id,role,sender,timestamp,content
1,85111b57a85ba964acc33d44ae83786d,user,user,2024-02-02 23:48:27.865715,"{""instruction"": {""Task"": ""Think step by step, ..."


In [ ]:
# get_last_rows(sender=None, role=None, n=1)
df = branch.get_last_rows(n=3)
df

,node_id,role,sender,timestamp,content
2,48d4f75218f74f7a3a16c259e6be3bf1,assistant,action_request,2024-02-02 23:48:33.040024,"{""action_request"": [{""recipient_name"": ""functions..."
3,960089cac08d9cf4635f8639db9b1ba6,assistant,action_response,2024-02-02 23:48:33.041690,"{""action_response"": {""function"": ""multiply"", ""..."
4,868b9e265c6e5aa46b767efcc6a45a3d,assistant,action_response,2024-02-02 23:48:33.042977,"{""action_response"": {""function"": ""multiply"", ""..."


### Edit

In [ ]:
# change_first_system_message(system=None, sender=None)
branch.change_first_system_message(system='Updated System')

In [ ]:
# add_message(system=None, instruction=None, context=None, response=None, sender=None)
# system, instruction, response: only choose one
branch.add_message(instruction='a new instruction added')

In [ ]:
# replace_keyword(keyword, replacement, case_sensitive=False)
branch.replace_keyword('function', 'func')

In [ ]:
# rollback(steps)
branch.rollback(4)

In [ ]:
# extend(messsages:pd.DataFrame)
branch.extend(df)